https://www.youtube.com/watch?v=sZdIybqppqQ

In [ ]:
!pip install ktrain

     |████████████████████████████████| 25.3MB 137kB/s 
     |████████████████████████████████| 983kB 46.2MB/s 
     |████████████████████████████████| 266kB 49.5MB/s 
     |████████████████████████████████| 1.3MB 44.0MB/s 
     |████████████████████████████████| 471kB 41.0MB/s 
     |████████████████████████████████| 1.1MB 47.0MB/s 
     |████████████████████████████████| 2.9MB 47.8MB/s 
     |████████████████████████████████| 890kB 45.4MB/s 
  Created wheel for ktrain: filename=ktrain-0.25.0-cp36-none-any.whl size=25274500 sha256=7fb7b62402bde8ec0a84de2c9b90bea42afad918e66b930bc289962b42361158
  Stored in directory: /root/.cache/pip/wheels/5c/73/05/f36d0027bb6575384e21506dbba8db36a7825f15a24f09b2d5
  Created wheel for keras-bert: filename=keras_bert-0.86.0-cp36-none-any.whl size=34145 sha256=08a0a8a8d7d3ff20dfefb0d02b8571063710d8f63e07cf4d9d333bf20d2f706a
  Stored in directory: /root/.cache/pip/wheels/66/f0/b1/748128b58562fc9e31b907bb5e2ab6a35eb37695e83911236b
  Created wheel for lan

In [ ]:
!git clone https://github.com/laxmimerit/IMDB-Movie-Reviews-Large-Dataset-50k.git

Cloning into 'IMDB-Movie-Reviews-Large-Dataset-50k'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 10 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (10/10), done.


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import ktrain
from ktrain import text

In [ ]:
from google.colab import files 

uploadedtrain = files.upload()
#uploadedtest = files.upload()

Saving train.csv to train.csv


In [ ]:
import io
dftrain = pd.read_csv(io.BytesIO(uploadedtrain['train.csv']))
#dftest = pd.read_csv(io.BytesIO(uploadedtest['test.csv']))

# Dataset is now stored in a Pandas Dataframe

In [ ]:
#data_train = pd.read_excel('/content/IMDB-Movie-Reviews-Large-Dataset-50k/train.xlsx',dtype=str)
dftrain.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [ ]:
#data_test = pd.read_excel('/content/IMDB-Movie-Reviews-Large-Dataset-50k/test.xlsx',dtype=str)
#dftest.head()

In [ ]:
train=dftrain.sample(frac=0.92, random_state=52)
test=dftrain.loc[~dftrain.index.isin(train.index)]

In [ ]:
train.tail()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
110678,500798a3abc879e7,"the best \n\nHey I hear you're the best, don't...",0,0,0,0,0,0
139580,eb04b41bb5d41526,What is ladies' day? \n\nThe news today is men...,0,0,0,0,0,0
25651,43e6cb16a8d6dcf7,"""\n\nAhh the so called """"tranny chasers"""". To...",0,0,0,0,0,0
126599,a522c900e7ea6282,"""\n\n offline? \n\nhas anyone been able to acc...",0,0,0,0,0,0
39117,686182438c1b8c10,Further note to potential reviewing admin If w...,0,0,0,0,0,0


In [ ]:
test.tail()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
159491,feebe635d6e273c6,"""\n\n I've provided a citation that doesn't in...",0,0,0,0,0,0
159517,ff4eb629bb10208f,"""\nYou've got no call to start attacking me ad...",0,0,0,0,0,0
159521,ff5e13378ca9d7a7,Improv\nWhat is the name of the part where the...,0,0,0,0,0,0
159562,ffd72e9766c09c97,"""\n Auto guides and the motoring press are not...",0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0


In [ ]:
LABEL_COLUMNS = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']

In [ ]:
(x_train,y_train),(x_test,y_test),preprocess=text.texts_from_df(train_df=train,
                                                                text_column='comment_text',
                                                                label_columns=LABEL_COLUMNS,
                                                                val_df=test,
                                                                maxlen=410,
                                                                max_features=35000,
                                                                preprocess_mode = 'bert')

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? True
preprocessing test...
language: en


In [ ]:
x_train[0].shape

(146805, 410)

In [ ]:
model = text.text_classifier(name='bert',train_data=(x_train,y_train),preproc=preprocess)

Is Multi-Label? True
maxlen is 410
done.


In [ ]:
# Get Learning Rate
learner = ktrain.get_learner(model=model,train_data=(x_train,y_train),val_data=(x_test,y_test),
                             batch_size=12)

In [ ]:
# might take days to run
learner.lr_find(max_epochs=5)

# got lr from here and input to the next step
# could use 5e-5, 3e-5, or 2e-5 for test purpose

simulating training for different learning rates... this may take a few moments...
Epoch 1/5
12234/12234 [==============================] - 17610s 1s/step - loss: 0.1258 - accuracy: 0.7294
Epoch 2/5
12035/12234 [============================>.] - ETA: 4:46 - loss: 0.0528 - accuracy: 0.9090

In [ ]:
learner.lr_plot()

NameError: ignored

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
learner.fit_onecycle(lr=3e-5,epochs=1)
predictor = ktrain.get_predictor(learner.model,preprocess)
predictor.save('/content/bert_2epochs_1122')



begin training using onecycle policy with max lr of 3e-05...
12234/12234 [==============================] - 17676s 1s/step - loss: 0.0471 - accuracy: 0.9051 - val_loss: 0.0374 - val_accuracy: 0.9793


In [ ]:
# test model prediction
s1=['Damnit! wtf is that! ']
s2=['Hope you guys have a great time watching this.']
s3=['you big bug ars bastard.']

In [ ]:
predictor.predict(s1)

[[('toxic', 0.96804553),
  ('severe_toxic', 0.02544431),
  ('obscene', 0.83290935),
  ('threat', 0.0008973419),
  ('insult', 0.13064218),
  ('identity_hate', 0.0049895463)]]

In [ ]:
predictor.predict(s2)

[[('toxic', 0.00022489125),
  ('severe_toxic', 4.441971e-05),
  ('obscene', 4.195973e-05),
  ('threat', 5.9065813e-05),
  ('insult', 3.3841112e-05),
  ('identity_hate', 4.5670826e-05)]]

In [ ]:
predictor.predict(s3)

[[('toxic', 0.9966704),
  ('severe_toxic', 0.2108403),
  ('obscene', 0.9807787),
  ('threat', 0.002325417),
  ('insult', 0.95083505),
  ('identity_hate', 0.026234658)]]

In [ ]:
predictor.save("mymodel")

In [ ]:
from google.colab import auth
auth.authenticate_user()
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')
from googleapiclient.http import MediaFileUpload

file_metadata = {'name': 'tf_model_h5'}

media = MediaFileUpload('/content/bert_2epochs_1122/tf_model.h5', resumable=True)
created = drive_service.files().create(body=file_metadata,
media_body=media,
fields='id').execute()
print('File ID: {}'.format(created.get('id')))

File ID: 1K-UQH2c4HReWPby0S85EyZmZpAolfWV5
